In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
predf = pd.read_csv('../../data/usa_00004.csv.gz')

In [3]:
predf =  predf.loc[predf['PERNUM'] == 1]

In [4]:
predf = predf.drop(['SAMPLE', 'SERIAL', 'CBSERIAL', 'CLUSTER', 'COUNTYICP','STRATA','PERNUM_HEAD', 'GQ', 'RACED'], axis = 1)

In [5]:
predf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88979 entries, 0 to 210221
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   YEAR       88979 non-null  int64  
 1   HHWT       88979 non-null  float64
 2   STATEFIP   88979 non-null  int64  
 3   COUNTYFIP  88979 non-null  int64  
 4   PERNUM     88979 non-null  int64  
 5   PERWT      88979 non-null  float64
 6   SEX        88979 non-null  int64  
 7   AGE        88979 non-null  int64  
 8   MARST      88979 non-null  int64  
 9   RACE       88979 non-null  int64  
 10  HCOVANY    88979 non-null  int64  
 11  EDUC       88979 non-null  int64  
 12  EDUCD      88979 non-null  int64  
 13  EMPSTAT    88979 non-null  int64  
 14  EMPSTATD   88979 non-null  int64  
 15  FTOTINC    88979 non-null  int64  
dtypes: float64(2), int64(14)
memory usage: 11.5 MB


In [6]:
predf.EMPSTATD.value_counts()

10    48919
30    36357
20     1957
12     1096
14      487
0       160
15        3
Name: EMPSTATD, dtype: int64

In [7]:
predf['COUNTYFIP'] = predf.COUNTYFIP.astype(str).str.zfill(3)

In [8]:
predf.HHWT = round(predf['HHWT']).astype(int)

In [9]:
df = predf

In [10]:
def YearStateDF(df, year, statefip):
    dfstate = df.loc[(df['YEAR'] == year) & (df['STATEFIP'] == statefip)]
    dfstatewt = dfstate.loc[dfstate.index.repeat(dfstate.HHWT)]
    newdf = dfstatewt.set_index('COUNTYFIP')
    return newdf

In [11]:
def NumericDF(df):
    inc = pd.DataFrame(df.groupby('COUNTYFIP')['FTOTINC'].mean().round(decimals=2))
    df['avg_inc'] = inc['FTOTINC']
    age = pd.DataFrame(df.groupby('COUNTYFIP')['AGE'].mean().round(decimals=2))
    df['avg_age'] = age['AGE']
    educ = pd.DataFrame(df.groupby('COUNTYFIP')['EDUC'].mean().round(decimals=2))
    df['avg_educ'] = educ['EDUC']
    educd = pd.DataFrame(df.groupby('COUNTYFIP')['EDUCD'].mean().round(decimals=2))
    df['avg_educd'] = educd['EDUCD']
    #ins = pd.DataFrame(df.groupby('COUNTYFIP')['HCOVANY'].mean().round(decimals=2))
    #df['avg_ins'] = ins['HCOVANY']
    df.drop(['FTOTINC', 'AGE', 'EDUC', 'EDUCD'], axis = 1)
    return df

In [ ]:
def catmap(df, column, dict):
    df[column].map(dict)
    dfcatmap = df.reset_index().groupby(['COUNTYFIP', column]).count().reset_index().pivot(columns=column, 
                                                                                         index='COUNTYFIP',
                                                                                        values='YEAR')
    return dfcatmap

In [12]:
pd.DataFrame(df.groupby(['STATEFIP', 'COUNTYFIP']).count().T.sum()).reset_index().groupby('STATEFIP').sum()

,0
STATEFIP,
6,412720
12,252756
17,152670
26,122626
48,304934


In [13]:
df.STATEFIP.unique()

array([ 6, 12, 17, 26, 48])

In [14]:
#df.STATEFIP.value_counts()

In [15]:
TX2015 = YearStateDF(df, 2015, 48)
TX2015

,YEAR,HHWT,STATEFIP,PERNUM,PERWT,SEX,AGE,MARST,RACE,HCOVANY,EDUC,EDUCD,EMPSTAT,EMPSTATD,FTOTINC
COUNTYFIP,,,,,,,,,,,,,,,
113,2015,3593,48,1,3592.80,2,69,5,1,2,7,71,3,30,125200
113,2015,3593,48,1,3592.80,2,69,5,1,2,7,71,3,30,125200
113,2015,3593,48,1,3592.80,2,69,5,1,2,7,71,3,30,125200
113,2015,3593,48,1,3592.80,2,69,5,1,2,7,71,3,30,125200
113,2015,3593,48,1,3592.80,2,69,5,1,2,7,71,3,30,125200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,2015,2949,48,1,2964.06,2,69,4,1,2,0,2,3,30,11400
215,2015,2949,48,1,2964.06,2,69,4,1,2,0,2,3,30,11400
215,2015,2949,48,1,2964.06,2,69,4,1,2,0,2,3,30,11400


In [16]:
#TX2015 = YearStateDf(df, 2015, 48)
#TX2015

In [17]:
TX2015 = NumericDF(TX2015)
TX2015

,YEAR,HHWT,STATEFIP,PERNUM,PERWT,SEX,AGE,MARST,RACE,HCOVANY,EDUC,EDUCD,EMPSTAT,EMPSTATD,FTOTINC,avg_inc,avg_age,avg_educ,avg_educd
COUNTYFIP,,,,,,,,,,,,,,,,,,,
113,2015,3593,48,1,3592.80,2,69,5,1,2,7,71,3,30,125200,341840.03,47.51,7.02,72.55
113,2015,3593,48,1,3592.80,2,69,5,1,2,7,71,3,30,125200,341840.03,47.51,7.02,72.55
113,2015,3593,48,1,3592.80,2,69,5,1,2,7,71,3,30,125200,341840.03,47.51,7.02,72.55
113,2015,3593,48,1,3592.80,2,69,5,1,2,7,71,3,30,125200,341840.03,47.51,7.02,72.55
113,2015,3593,48,1,3592.80,2,69,5,1,2,7,71,3,30,125200,341840.03,47.51,7.02,72.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,2015,2949,48,1,2964.06,2,69,4,1,2,0,2,3,30,11400,392355.49,49.49,5.66,58.97
215,2015,2949,48,1,2964.06,2,69,4,1,2,0,2,3,30,11400,392355.49,49.49,5.66,58.97
215,2015,2949,48,1,2964.06,2,69,4,1,2,0,2,3,30,11400,392355.49,49.49,5.66,58.97


In [18]:
TX2015['RACE'].value_counts()

1    7636711
2    1358455
7     402931
6     373068
8     168798
4      75555
3      59655
9      12695
5       7081
Name: RACE, dtype: int64

In [19]:
predf.EMPSTATD.value_counts()

10    48919
30    36357
20     1957
12     1096
14      487
0       160
15        3
Name: EMPSTATD, dtype: int64

In [32]:
categoricals

['SEX', 'MARST', 'RACE', 'RACED', 'EMPSTAT', 'EMPSTATD']

In [20]:
emp_mapper = {0:'NA',
10:'at_work',
12:'has_job_not_working',
14:'armed_forces_at_work',
15:'armed_forces_not_at_work_but_with_job',
20:'unemployed',
30:'not_in_labor_force'}

In [21]:
def catmap(df, column, dict):
    df[column].map(dict)
    dfcatmap = df.reset_index().groupby(['COUNTYFIP', column]).count().reset_index().pivot(columns=column, 
                                                                                         index='COUNTYFIP',
                                                                                        values='YEAR')
    return dfcatmap

In [22]:
txempstatd = catmap(TX2015, 'EMPSTATD', emp_mapper)
txempstatd

EMPSTATD,0,10,12,14,15,20,30
COUNTYFIP,,,,,,,
000,5899.0,990169.0,17425.0,7739.0,449.0,51151.0,847334.0
027,NaN,74821.0,1078.0,15659.0,NaN,10838.0,23548.0
029,1332.0,395391.0,9252.0,13308.0,NaN,19132.0,188857.0
039,NaN,104339.0,NaN,NaN,NaN,NaN,34176.0
041,NaN,45776.0,NaN,NaN,NaN,1168.0,34565.0
061,NaN,79460.0,NaN,NaN,NaN,2245.0,59206.0
085,NaN,246302.0,210.0,1976.0,NaN,5957.0,58139.0
091,NaN,37395.0,NaN,NaN,NaN,838.0,21708.0
113,150.0,676713.0,11617.0,NaN,NaN,15510.0,285437.0


In [23]:
race_mapper = {1:'White',
2:'Black/African American/Negro',
3:'American Indian or Alaska Native',
4:'Chinese',
5:'Japanese',
6:'Other Asian or Pacific Islander',
7:'Other race, nec',
8:'Two major races',
9:'Three or more major races',}

TX2015['RACE'].map(race_mapper)

COUNTYFIP
113    White
113    White
113    White
113    White
113    White
       ...  
215    White
215    White
215    White
215    White
215    White
Name: RACE, Length: 10094949, dtype: object

In [24]:
TXbyRACE = TX2015.reset_index().groupby(['COUNTYFIP', 'RACE']).count().reset_index().pivot(columns='RACE', 
                                                                                         index='COUNTYFIP',
                                                                                        values='YEAR')
TXbyRACE

RACE,1,2,3,4,5,6,7,8,9
COUNTYFIP,,,,,,,,,
000,1604633.0,193598.0,10734.0,4236.0,NaN,8802.0,61665.0,34193.0,2305.0
027,94654.0,20975.0,509.0,NaN,NaN,4835.0,2426.0,2545.0,NaN
029,512474.0,51163.0,1437.0,3518.0,1183.0,16226.0,24595.0,16676.0,NaN
039,97108.0,28174.0,NaN,NaN,NaN,8368.0,2934.0,1931.0,NaN
041,61316.0,9369.0,NaN,2305.0,NaN,554.0,3504.0,4461.0,NaN
061,132603.0,NaN,NaN,NaN,NaN,NaN,7320.0,988.0,NaN
085,237273.0,21945.0,3952.0,8218.0,NaN,29116.0,10074.0,2006.0,NaN
091,51242.0,2201.0,NaN,NaN,NaN,2141.0,1243.0,1692.0,1422.0
113,609354.0,244681.0,6272.0,8308.0,NaN,58968.0,43579.0,14896.0,3369.0


In [25]:
marital_mapper ={1:'married_spouse_present',
2:'married_spouse_absent',
3:'separated',
4:'divorced',
5:'widowed',
6:'single'}
TX2015['MARST'].map(marital_mapper)

COUNTYFIP
113     widowed
113     widowed
113     widowed
113     widowed
113     widowed
         ...   
215    divorced
215    divorced
215    divorced
215    divorced
215    divorced
Name: MARST, Length: 10094949, dtype: object

In [26]:
TX2015['MARST'].value_counts()

1    4815589
6    2203573
4    1640756
5     789427
3     354162
2     291442
Name: MARST, dtype: int64

In [27]:
TXMARST = TX2015.reset_index().groupby(['COUNTYFIP', 'MARST']).count().reset_index().pivot(columns='MARST', 
                                                                                         index='COUNTYFIP',
                                                                                        values='YEAR')
TXMARST

MARST,1,2,3,4,5,6
COUNTYFIP,,,,,,
000,919875.0,70559.0,69643.0,312739.0,201655.0,345695.0
027,64373.0,16033.0,3413.0,17679.0,2275.0,22171.0
029,262643.0,15032.0,16391.0,123468.0,49341.0,160397.0
039,60164.0,5868.0,1018.0,39445.0,7051.0,24969.0
041,19610.0,NaN,NaN,13174.0,9057.0,39668.0
061,87381.0,3593.0,2619.0,9295.0,18801.0,19222.0
085,201990.0,2619.0,8682.0,38412.0,13952.0,46929.0
091,35554.0,1437.0,NaN,13039.0,2036.0,7875.0
113,410208.0,34072.0,53518.0,129412.0,75207.0,287010.0


In [28]:
numerics = ['AGE', 'EDUC', 'EDUCD', 'FTOTINC']
categoricals = ['SEX', 'MARST','RACE','RACED', 'EMPSTAT', 'EMPSTATD']
binary = ['HCOVANY','SEX']

In [29]:
#I need to make 2 functions
#one that makes a dataframe by state and year
#one that makes a new dataframe grouped by colums with means of numerical data and dummies of categorical data?

In [30]:
#def YearStateDf(df, year, statefip):
 #   dfstate = df.loc[(df['YEAR'] == year) & (df['STATEFIP'] == statefip)]
    #dfstatewt = dfstate.loc[dfstate.index.repeat(dfstate.HHWT)]
    #dfstatewt.set_index('COUNTYFIP')
    #return dfstatewt

In [31]:
#def Numeric(df):
    #inc = pd.DataFrame(df.groupby('COUNTYFIP')['FTOTINC'].mean().round(decimals=2))
    #df['avg_inc'] = inc['FTOTINC']
    #age = pd.DataFrame(df.groupby('COUNTYFIP')['AGE'].mean()).round(decimals=2))
    #df['avg_age'] = age['AGE']
    #return df